In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
conn = psycopg2.connect(dbname='airbnb_project', user='postgres', host='localhost', password='123456', port="5432")

In [3]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df

In [5]:
# Carregue os dados das tabelas 'listings' , 'reviews' , 'calendar'.
df_listings = load_data_from_table('listings')
df_reviews = load_data_from_table('reviews')
df_calendar = load_data_from_table('calendar')

C:\Users\Acer\AppData\Local\Temp\ipykernel_1816\2710205896.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [13]:
df_listings
#df_listings.columns

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20231226034138,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,None,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,...,4.91,4.77,4.67,None,False,1,1,0,0,1.90
1,25026,https://www.airbnb.com/rooms/25026,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.72 · 1 bedr...,None,Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,102840,...,4.92,4.84,4.60,None,False,1,1,0,0,1.67
2,35764,https://www.airbnb.com/rooms/35764,20231226034138,2023-12-27,city scrape,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,None,Our guests will experience living with a local...,https://a0.muscache.com/pictures/23782972/1d3e...,153691,...,4.95,4.94,4.89,None,False,1,1,0,0,2.82
3,41198,https://www.airbnb.com/rooms/41198,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.21 · 2 bedr...,None,None,https://a0.muscache.com/pictures/3576716/2d6a9...,178975,...,4.56,4.44,4.38,None,False,2,2,0,0,0.13
4,326205,https://www.airbnb.com/rooms/326205,20231226034138,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.57 · 1 bedroom · ...,None,None,https://a0.muscache.com/pictures/c550151d-910c...,1603206,...,4.77,4.83,4.59,None,False,5,5,0,0,1.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36003,1053746406056980189,https://www.airbnb.com/rooms/1053746406056980189,20231226034138,2023-12-26,city scrape,Home in Rio de Janeiro · ★New · 1 bedroom · 4 ...,None,None,https://a0.muscache.com/pictures/86b7323c-41eb...,470173166,...,NaN,NaN,NaN,None,False,8,2,5,1,NaN
36004,1053756202332288557,https://www.airbnb.com/rooms/1053756202332288557,20231226034138,2023-12-26,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,None,Copacabana is located in Rio de Janeiro's stun...,https://a0.muscache.com/pictures/prohost-api/H...,6000862,...,NaN,NaN,NaN,None,True,185,185,0,0,NaN
36005,1053789340172837654,https://www.airbnb.com/rooms/1053789340172837654,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,None,None,https://a0.muscache.com/pictures/miso/Hosting-...,206898000,...,NaN,NaN,NaN,None,False,1,0,1,0,NaN
36006,1053808194231554793,https://www.airbnb.com/rooms/1053808194231554793,20231226034138,2023-12-28,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,None,None,https://a0.muscache.com/pictures/hosting/Hosti...,536983374,...,NaN,NaN,NaN,None,False,1,1,0,0,NaN


In [14]:
df_reviews
#df_reviews.columns

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,216700,1511171,2012-06-18,2598011,Evelyn,"A estada foi maravilhosa, ficamos so 3 noites ..."
2,216700,1550596,2012-06-24,2465739,Camila,The apartment is located in the beautiful Lara...
3,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
4,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
...,...,...,...,...,...,...
703791,1053655074004481557,1055764873322213198,2023-12-27,523745376,Adriana,"Apartamento top! Super novo, limpinho, todo eq..."
703792,1053716797020259779,1054357802054468817,2023-12-25,488321805,Juliana,Passamos dias maravilhosos aqui! A vista é sim...
703793,1053716797020259779,1055794588292689437,2023-12-27,206634401,Victor,Excelente anfitrião! Super atencioso e cortês!...
703794,1050572397352975357,1055147776405273833,2023-12-26,257578099,Dominiky,"Li preparou tudo e nós recebeu muito bem, com ..."


In [15]:
df_calendar
#df_calendar.columns

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-12-27,False,$350.00,None,5.0,28.0
1,17878,2023-12-28,False,$350.00,None,5.0,28.0
2,17878,2023-12-29,False,$350.00,None,5.0,28.0
3,17878,2023-12-30,False,$350.00,None,5.0,28.0
4,17878,2023-12-31,False,$350.00,None,5.0,28.0
...,...,...,...,...,...,...,...
13145590,1053155096258508813,2024-12-20,False,$250.00,None,3.0,7.0
13145591,1053155096258508813,2024-12-21,False,$250.00,None,3.0,7.0
13145592,1053155096258508813,2024-12-22,False,$250.00,None,3.0,7.0
13145593,1053155096258508813,2024-12-23,False,$250.00,None,3.0,7.0


# Silver

## clean data

In [ ]:
def remove_valores_ausentes(df):
    pass

In [ ]:
def remove_duplicatas(df):
    pass

In [ ]:
def remove_outliers(df):
    pass

In [12]:
#def clean_data(df):
    # Identificar e lidar com valores ausentes
    #df = df.replace('', np.nan)  # substituir strings vazias por NaN
    #df = df.dropna()  # remover linhas com valores NaN

    # Lidar com duplicatas
    #df = df.drop_duplicates()

    # Lidar com outliers
    # Aqui, usamos o método do Intervalo Interquartil (IQR) para identificar e remover outliers
    # Este é apenas um exemplo e pode não ser adequado para todos os conjuntos de dados ou colunas
    #Q1 = df.quantile(0.25)
    #Q3 = df.quantile(0.75)
    #IQR = Q3 - Q1
    #df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

    #return df

In [ ]:
# Exemplo de uso:
#df_listings_clean = clean_data(df_listings)
#df_reviews_clean = clean_data(df_reviews)
#df_calendar_clean = clean_data(df_calendar)

## padronizacao de colunas

In [ ]:
def clean_column_names(df):
    # Remover espaços e converter para minúsculas
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

In [ ]:
def converte_formato(df):
    pass

In [24]:
def tratar_df_reviews(df):
    df['date'] = pd.to_datetime(df['date'])

    return df

In [25]:
df_reviews_clean = tratar_df_reviews(df_reviews)
df_reviews_clean.dtypes

listing_id                int64
id                        int64
date             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comments                 object
dtype: object

## Limpeza textual

In [ ]:
def remove_chars(df):
    pass

In [ ]:
def remove_erros(df):
    pass

In [ ]:
print(df_calendar.dtypes)

In [28]:
df_reviews_clean

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,216700,1511171,2012-06-18,2598011,Evelyn,"A estada foi maravilhosa, ficamos so 3 noites ..."
2,216700,1550596,2012-06-24,2465739,Camila,The apartment is located in the beautiful Lara...
3,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
4,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
...,...,...,...,...,...,...
703791,1053655074004481557,1055764873322213198,2023-12-27,523745376,Adriana,"Apartamento top! Super novo, limpinho, todo eq..."
703792,1053716797020259779,1054357802054468817,2023-12-25,488321805,Juliana,Passamos dias maravilhosos aqui! A vista é sim...
703793,1053716797020259779,1055794588292689437,2023-12-27,206634401,Victor,Excelente anfitrião! Super atencioso e cortês!...
703794,1050572397352975357,1055147776405273833,2023-12-26,257578099,Dominiky,"Li preparou tudo e nós recebeu muito bem, com ..."


In [ ]:
# df_listings = clean_column_names(df_listings)
# df_reviews = clean_column_names(df_reviews)
# df_calendar = clean_column_names(df_calendar)

In [ ]:
#def clean_data_specific(df):
#    for col in df.columns:
#        if df[col].dtype == 'object':  # se a coluna é do tipo texto
#            df[col] = df[col].apply(clean_text)
#            Se a coluna deve ser numérica, converta-a
#            if col in ['host_response_rate', 'host_acceptance_rate', 'price']:
                #df[col] = df[col].str.replace('%', '').str.replace('$', '').astype(float)
#        elif df[col].dtype in ['int64', 'float64']:  # se a coluna é numérica
#            df[col] = df[col].fillna(0)  # preenche valores ausentes com 0
#        elif df[col].dtype == 'datetime64':  # se a coluna é uma data/hora
#           df[col] = pd.to_datetime(df[col], errors='coerce')  # converte para o tipo de data correto
#    return df

In [ ]:
# Exemplo de uso:
# df = clean_data_specific(df)